<a href="https://colab.research.google.com/github/prithwis/AGI/blob/main/Chandrayaan_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![CC-BY-SA](https://licensebuttons.net/l/by-sa/3.0/88x31.png)<br>


![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>




[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

# ChandraYaan

In [2]:
# 1. Update package list and install the NEW opengl names
!apt-get update > /dev/null 2>&1
!apt-get install -y xvfb ffmpeg freeglut3-dev python3-opengl libgl1-mesa-dev libglu1-mesa-dev mesa-utils > /dev/null 2>&1

# 2. Install the system dependencies (SWIG is the key here)
#!apt-get update
!apt-get install -y swig build-essential python3-dev > /dev/null 2>&1

# 3. Upgrade pip and setuptools to handle the build process better
!pip install --upgrade pip setuptools wheel > /dev/null 2>&1

# 4. Now install gymnasium with box2d support
!pip install "gymnasium[box2d]" > /dev/null 2>&1

# 5. Install the Python libraries
!pip install pyvirtualdisplay  pygame opencv-python > /dev/null 2>&1


In [3]:
# 1. Install dependencies (Quiet mode)
#!pip install "gymnasium[box2d]" > /dev/null 2>&1

# 2. Verify installation
import gymnasium as gym
try:
    env = gym.make("LunarLander-v3")
    print("🚀 Lunar Lander is ready for takeoff!")
    env.close()
except Exception as e:
    print(f"❌ Installation issue: {e}")
    print("Try: Menu -> Runtime -> Restart Session")

/usr/local/lib/python3.12/dist-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


🚀 Lunar Lander is ready for takeoff!


In [4]:
import pygame
import cv2
import numpy as np
import os
import numpy as np
import random

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import gymnasium as gym
import cv2
import numpy as np
import os

from IPython.display import Video, display


In [5]:
import cv2
import numpy as np
import os

class VideoRecorder:
    def __init__(self, filename='simulation.avi', width=400, height=400, fps=15):
        self.filename = filename
        self.width = width
        # Add 60 pixels to the height for the dedicated dashboard area
        self.dash_height = 60
        self.total_height = height + self.dash_height
        self.fps = fps
        self.fourcc = cv2.VideoWriter_fourcc(*'XVID')
        self.video_writer = None

    def start(self):
        if os.path.exists(self.filename):
            os.remove(self.filename)
        # Crucial: The dimensions here MUST match the frames we write later
        self.video_writer = cv2.VideoWriter(
            self.filename, self.fourcc, self.fps, (self.width, self.total_height)
        )

    def record_frame_with_hud(self, frame_array, reward, step):
        # 1. Prepare the environment frame (the 7x7 grid)
        view = cv2.resize(frame_array, (self.width, self.total_height - self.dash_height))
        # Ensure it is in BGR for OpenCV
        #if len(view.shape) == 3 and view.shape[2] == 3:
        #    view = cv2.cvtColor(view, cv2.COLOR_RGB2BGR)

        # 2. Create the light-colored Dashboard (the background for text)
        # BGR (240, 240, 240) is a nice light gray
        dashboard = np.full((self.dash_height, self.width, 3), 240, dtype=np.uint8)

        # 3. Add SMALLER, DARKER text to the dashboard
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.45  # Small and crisp
        thickness = 1
        text_color = (60, 60, 60) # Dark Gray

        cv2.putText(dashboard, f"STEP: {step}", (15, 20),
                    font, font_scale, text_color, thickness, cv2.LINE_AA)
        cv2.putText(dashboard, f"CUMULATIVE REWARD: {reward}", (15, 45),
                    font, font_scale, text_color, thickness, cv2.LINE_AA)

        # 4. Stack them: Dashboard on TOP, Environment on BOTTOM
        combined_frame = np.vstack((dashboard, view))

        # 5. Write to file
        self.video_writer.write(combined_frame)

    def stop(self):
        if self.video_writer:
            self.video_writer.release()

        output_mp4 = self.filename.replace('.avi', '.mp4')
        # Final conversion for browser compatibility
        os.system(f"ffmpeg -y -i {self.filename} -c:v libx264 -pix_fmt yuv420p {output_mp4} -hide_banner -loglevel error")
        return output_mp4

In [6]:
import gymnasium as gym

# Initialize the environment
# 'rgb_array' is required so your VideoRecorder can "see" the frames
env = gym.make("LunarLander-v3", render_mode="rgb_array")

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
from collections import deque
import numpy as np

# 1. THE BRAIN (The architecture you used successfully before)
class PowerhouseDQN(nn.Module):
    def __init__(self, state_size, action_size):
        super(PowerhouseDQN, self).__init__()
        self.fc1 = nn.Linear(state_size, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, action_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x) # Linear output for Q-values


In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
from collections import deque
import numpy as np


# 2. THE AGENT (The Pilot using the Stable Target Network)
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=20000) # Increased memory for more "experience"
        self.gamma = 0.99
        self.epsilon = 1.0
        self.epsilon_min = 0.05
        self.epsilon_decay = 0.996
        self.learning_rate = 0.0005 # Stable learning rate

        # Main Model (the one being trained every step)
        self.model = PowerhouseDQN(state_size, action_size)

        # Target Model (the stable one used for predictions)
        self.target_model = PowerhouseDQN(state_size, action_size)
        self.update_target_model() # Sync them initially

        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)

    def update_target_model(self):
        """Copies weights from model to target_model"""
        self.target_model.load_state_dict(self.model.state_dict())

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        state = torch.FloatTensor(state).unsqueeze(0)
        with torch.no_grad():
            act_values = self.model(state)
        return torch.argmax(act_values[0]).item()

    def replay(self, batch_size):
        if len(self.memory) < batch_size:
            return

        minibatch = random.sample(self.memory, batch_size)

        for state, action, reward, next_state, done in minibatch:
            state_t = torch.FloatTensor(state).unsqueeze(0)
            next_state_t = torch.FloatTensor(next_state).unsqueeze(0)

            target = reward
            if not done:
                # Use target_model for the next-state prediction (Stability!)
                target = (reward + self.gamma * torch.max(self.target_model(next_state_t)[0]).item())

            target_f = self.model(state_t)
            target_f[0][action] = target

            self.optimizer.zero_grad()
            loss = F.mse_loss(self.model(state_t), target_f)
            loss.backward()
            self.optimizer.step()

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

# Initialize the agent
#agent = DQNAgent(state_size=8, action_size=4)

#Tuning to prevent hover

In [ ]:
!rm dqn*.*

rm: cannot remove 'dqn*.*': No such file or directory


In [9]:
import numpy as np

agentY = DQNAgent(8, 4)

episodes = 500
#batch_size = 32
MAX_STEPS = 600 # Safety cap per flight

for e in range(episodes):
    state, info = env.reset()
    state = np.reshape(state, [8])
    total_reward = 0

    # 1. ONLY start the recorder on specific milestones
    should_record = (e == 0 or e == episodes - 1)

    if should_record:
        print(f"🎬 Recording Milestone Episode: {e}")
        recorder = VideoRecorder(filename=f'dqn_milestone_{e}.avi', width=600, height=400)
        recorder.start()

    for step_count in range(MAX_STEPS):
        action = agentY.act(state)
        observation, reward, terminated, truncated, info = env.step(action)

        # Manually punish hovering too high (y is the 2nd element in the observation)
        y_pos = observation[1]
        if y_pos > 1.0:
            reward -= 2.0  # Apply a "stay on screen" penalty

        if action == 2: # Main Engine
            reward -= 0.5 # Increase the cost of firing the big engine

        done = terminated or truncated
        next_state = np.reshape(observation, [8])

        # Inside your loop, after the 'step_count' for loop ends:
        if not (terminated or truncated):
            # This means it timed out while hovering!
            reward -= 100
        #    agent.remember(state, action, reward, next_state, True)

        # Agent learns every step, even when not recording
        agentY.remember(state, action, reward, next_state, done)
        state = next_state
        total_reward += reward

        # 2. ONLY record frames if we are in a milestone episode
        if should_record:
            frame_rgb = env.render()
            frame_bgr = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR)
            recorder.record_frame_with_hud(frame_bgr, round(total_reward, 2), step_count)

        if done:
            break

    # 3. ONLY stop/convert the video if we were recording
    if should_record:
        recorder.stop()
        print(f"✅ Video for Episode {e} saved.")

    # Experience Replay (The Brain Update)
    agentY.replay(64)

    if e % 50 == 0:
        print(f"Episode: {e}/{episodes} | Score: {total_reward:.2f} | Epsilon: {agentY.epsilon:.2f}")

env.close()

🎬 Recording Milestone Episode: 0
✅ Video for Episode 0 saved.
Episode: 0/500 | Score: -10328.91 | Epsilon: 1.00
Episode: 50/500 | Score: -8436.49 | Epsilon: 0.82
Episode: 100/500 | Score: -10180.07 | Epsilon: 0.67
Episode: 150/500 | Score: -8308.42 | Epsilon: 0.55
Episode: 200/500 | Score: -7920.49 | Epsilon: 0.45
Episode: 250/500 | Score: -28926.36 | Epsilon: 0.37
Episode: 300/500 | Score: -9499.49 | Epsilon: 0.30
Episode: 350/500 | Score: -34169.78 | Epsilon: 0.24
Episode: 400/500 | Score: -10407.15 | Epsilon: 0.20
Episode: 450/500 | Score: -8133.04 | Epsilon: 0.16
🎬 Recording Milestone Episode: 499
✅ Video for Episode 499 saved.


In [10]:
#display(Video(first_flight, embed=True))
display(Video("/content/dqn_milestone_0.mp4", embed=True))
#display(Video(first_flight, embed=True))
display(Video("/content/dqn_milestone_499.mp4", embed=True))

In [11]:
!rm dqn*.*

In [12]:
import numpy as np

agentY = DQNAgent(8, 4)

episodes = 500
#batch_size = 32
MAX_STEPS = 600 # Safety cap per flight

for e in range(episodes):
    state, info = env.reset()
    state = np.reshape(state, [8])
    total_reward = 0

    should_record = (e == 0 or e == episodes - 1)
    if should_record:
        recorder = VideoRecorder(filename=f'dqn_milestone_{e}.avi', width=600, height=400)
        recorder.start()

    for step_count in range(MAX_STEPS):
        action = agentY.act(state)
        observation, reward, terminated, truncated, info = env.step(action)

        # --- SALVAGED REWARD SHAPING (Fixed) ---
        # 1. Fuel penalty: Only if it's really hovering too long
        if action == 2: reward -= 0.1 # Reduced from 0.5 to be less discouraging

        # 2. Out of bounds penalty
        if observation[1] > 1.0: reward -= 1.0

        done = terminated or truncated
        next_state = np.reshape(observation, [8])

        # --- THE FIX: Penalty moved OUTSIDE or triggered only on DONE ---
        if truncated and not terminated: # If it timed out without landing
            reward -= 100

        agentY.remember(state, action, reward, next_state, done)
        state = next_state
        total_reward += reward

        if should_record:
            frame_bgr = cv2.cvtColor(env.render(), cv2.COLOR_RGB2BGR)
            recorder.record_frame_with_hud(frame_bgr, round(total_reward, 2), step_count)

        if done:
            break

    if should_record:
        recorder.stop()

    # --- BRAIN UPDATES ---
    # Training the large 512/256 network needs more "study sessions"
    # per flight. We run replay 10 times.
    for _ in range(10):
        agentY.replay(64)

    # Sync the stable brain
    if e % 10 == 0:
        agentY.update_target_model()

    if e % 10 == 0:
        print(f"Episode: {e}/{episodes} | Score: {total_reward:.2f} | Epsilon: {agentY.epsilon:.2f}")

Episode: 0/500 | Score: -172.24 | Epsilon: 0.96
Episode: 10/500 | Score: -312.80 | Epsilon: 0.64
Episode: 20/500 | Score: -290.39 | Epsilon: 0.43
Episode: 30/500 | Score: -464.17 | Epsilon: 0.29
Episode: 40/500 | Score: -508.03 | Epsilon: 0.19
Episode: 50/500 | Score: -609.25 | Epsilon: 0.13
Episode: 60/500 | Score: -355.55 | Epsilon: 0.09
Episode: 70/500 | Score: -742.98 | Epsilon: 0.06
Episode: 80/500 | Score: -342.63 | Epsilon: 0.05
Episode: 90/500 | Score: -301.88 | Epsilon: 0.05
Episode: 100/500 | Score: -461.20 | Epsilon: 0.05
Episode: 110/500 | Score: 217.67 | Epsilon: 0.05
Episode: 120/500 | Score: -190.30 | Epsilon: 0.05
Episode: 130/500 | Score: -681.99 | Epsilon: 0.05
Episode: 140/500 | Score: -698.23 | Epsilon: 0.05
Episode: 150/500 | Score: -708.54 | Epsilon: 0.05
Episode: 160/500 | Score: -677.99 | Epsilon: 0.05
Episode: 170/500 | Score: -701.98 | Epsilon: 0.05
Episode: 180/500 | Score: -645.04 | Epsilon: 0.05
Episode: 190/500 | Score: -670.89 | Epsilon: 0.05
Episode: 200

In [13]:
#display(Video(first_flight, embed=True))
display(Video("/content/dqn_milestone_0.mp4", embed=True))
#display(Video(first_flight, embed=True))
display(Video("/content/dqn_milestone_499.mp4", embed=True))

#Chronobooks <br>
Three science fiction novels by Prithwis Mukerjee. A dystopian Earth. A technocratic society managed by artificial intelligence. Escape and epiphany on Mars. Can man and machine, carbon and silicon explore and escape into other dimensions of existence? An Indic perspective rooted in Advaita Vedanta and the Divine Feminine.  [More information](http://bit.ly/chrono3) <br>
![alt text](https://blogger.googleusercontent.com/img/a/AVvXsEjsZufX_KYaLwAnJP6bUxvDg5RSPn6r8HIZe749nLWX3RuwyshrYEAUpdw03a9WIWRdnzA9epwJOE05eDJ0Ad7kGyfWiUrC2vNuOskb2jA-e8aOZSx8YqzT8mfZi3E4X1Rz3qlEAiv-aTxlCM976BEeTjx4J64ctY3C_FoV4v9aY_U23F8xRqI5Eg=s1600)